# Make Granule List

In [1]:
# import utility function for making granule list
from icelakes.nsidc import make_granule_list

In [2]:
# # function to convert shapefile to geojson - if needed
# shp2geojson_nsidc('shapefiles/jakobshavn_small.shp')

In [3]:
# set parameters and make the list
geojson = 'jakobshavn_small.geojson'
start_date = '2019-07-01'
end_date = '2019-07-31'
outname = 'granule_lists/' + geojson.replace('.geojson','') + '_' + start_date[:4] + '.csv'

make_granule_list(geojson, start_date, end_date, outname)

Found 6 ATL03 version 005 granules over jakobshavn_small.geojson between 2019-07-01 and 2019-07-31.
Wrote file: granule_lists/jakobshavn_small_2019.csv


In [4]:
# make sure that it worked by reading the file into a DataFrame and displaying it
import pandas as pd
pd.read_csv(outname,header=None)

,0,1,2
0,ATL03_20190712052659_02160403_005_01.h5,geojsons/jakobshavn_small.geojson,jakobshavn_small-2019
1,ATL03_20190712181153_02240405_005_01.h5,geojsons/jakobshavn_small.geojson,jakobshavn_small-2019
2,ATL03_20190716051841_02770403_005_01.h5,geojsons/jakobshavn_small.geojson,jakobshavn_small-2019
3,ATL03_20190716180335_02850405_005_01.h5,geojsons/jakobshavn_small.geojson,jakobshavn_small-2019
4,ATL03_20190720051023_03380403_005_01.h5,geojsons/jakobshavn_small.geojson,jakobshavn_small-2019
5,ATL03_20190724050206_03990403_005_01.h5,geojsons/jakobshavn_small.geojson,jakobshavn_small-2019
